# TP : Classification de Texte avec Scikit-Learn

Dans ce TP, nous allons explorer les outils principaux de Scikit-Learn pour une tâche de classification de documents textuels, en utilisant le dataset des *Twenty Newsgroups*.

Objectifs :

- Charger et préparer les données
- Extraire des caractéristiques des textes
- Entraîner un modèle de classification
- Évaluer les performances du modèle

Nous aborderons aussi la recherche de paramètres optimaux via une recherche en grille.


### Préparation

Pour commencer, assurez-vous d'avoir installé *scikit-learn* et les autres dépendances nécessaires.

Commençons par charger les données et définir les catégories que nous utiliserons dans cet exercice.

> Question 1 : Importez les bibliothèques nécessaires et configurez le dataset pour contenir les catégories `alt.atheism`, `soc.religion.christian`, `comp.graphics`, et `sci.med`.

In [1]:
from sklearn.datasets import fetch_20newsgroups

categories = [
    "alt.atheism",
    "soc.religion.christian",
    "comp.graphics",
    "sci.med",
]

dataset = fetch_20newsgroups(categories=categories, shuffle=True, random_state=0)

### Analyse des Données Chargées

Les données sont chargées sous forme d'un *bunch*, un objet contenant plusieurs attributs utiles pour notre analyse.

> Question 2 : Affichez le nombre de documents et les catégories de nouvelles chargées dans le dataset.

In [11]:
print(len(dataset.data))
print(dataset["target_names"])


2257
['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']


### Exploration des Données Textuelles

> Question 3 : Affichez les premières lignes du premier document, ainsi que sa catégorie associée.

Cela nous donnera un aperçu des documents et de leur contenu.

In [21]:
print(dataset.data[0])
print(dataset.target_names[dataset.target[0]])

From: reedr@cgsvax.claremont.edu
Subject: Re: DID HE REALLY RISE???
Organization: The Claremont Graduate School
Lines: 29

In article <Apr.9.01.11.16.1993.16937@athos.rutgers.edu>, emery@tc.fluke.COM (John Emery) writes:
> The one single historic event that has had the biggest impact on the
> world over the centuries is the resurrection of Jesus.  At the same
> time, it is one of the most hotly contested topics....
> 
> Did Jesus Christ really rise from the dead?  Since the eyewitnesses
> are no longer living, we have only their written accounts. ...
> ...  Because of the magnitude of significance
> involved here, either the resurrection is the greatest event in the
> history of man or the greatest deception played on man.
> [massive amounts of data deleted]

John, 

While I will not take the time to rebut you point by point, I will suggest
three current works which I think will be helpful in your quest to answer
this question.  John Dominic Crossan (Professor of Religion at De Paul Un

In [25]:
len(dataset.data)

2257

### Extraction des Caractéristiques des Textes

Pour entraîner un modèle, nous devons transformer les documents textuels en vecteurs de caractéristiques numériques.

> Question 4 : Utilisez la méthode `CountVectorizer` pour transformer les textes en une matrice de fréquences de mots, puis affichez la taille de cette matrice.

Cette matrice, de type "sac de mots", représente chaque document par les occurrences des mots qu'il contient.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer()
X_counts = count_vectorizer.fit_transform(dataset.data)

In [23]:
type(X_counts)

scipy.sparse._csr.csr_matrix

In [24]:
X_counts.shape

(2257, 35788)

### Passage aux Fréquences Relatives (tf-idf)

Pour équilibrer l'importance des mots entre documents de différentes longueurs, on utilise la fréquence de termes (TF) ou bien TF-IDF.

> Question 5 : Transformez la matrice en représentation TF-IDF et affichez la dimension de cette nouvelle matrice.


In [26]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer()
X_tfidf = tfidf_transformer.fit_transform(X_counts)

In [27]:
X_tfidf.shape

(2257, 35788)

### Entraînement d'un Classifieur Naïve Bayes

Avec notre matrice de caractéristiques prête, entraînons un classifieur Naïve Bayes pour prédire la catégorie de chaque document.

> Question 6 : Entraînez un classifieur `MultinomialNB` sur le jeu de données transformé en TF-IDF.


In [28]:
from sklearn.naive_bayes import MultinomialNB

clf_nb = MultinomialNB()
clf_nb.fit(X_tfidf, dataset.target)

MultinomialNB()

### Prédiction sur des Nouveaux Documents

> Question 7 : Prédisez les catégories pour les phrases `"God is love"` et `"OpenGL on the GPU is fast"`.

Affichez la catégorie prédite pour chaque document.


In [29]:
documents = [
    "God is love",
    "OpenGL on the GPU is fast",
]

X_documents_tfidf = tfidf_transformer.transform(
    count_vectorizer.transform(documents)
)

clf_nb.predict(X_documents_tfidf)

array([3, 1], dtype=int64)

### Construction d'un Pipeline pour Simplifier le Workflow

> Question 8 : Créez un `Pipeline` qui enchaîne `CountVectorizer`, `TfidfTransformer` et `MultinomialNB`, puis entraînez le modèle sur le jeu de données.

Cela rend le processus d'extraction de caractéristiques et d'entraînement plus facile et modulaire.


In [31]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

pipeline_clf = Pipeline(
    steps=[
        ("vectorisation", TfidfVectorizer()),
        ("classification", MultinomialNB()),
    ]
)

In [32]:
pipeline_clf.fit(dataset.data, dataset.target)

Pipeline(steps=[('vectorisation', TfidfVectorizer()),
                ('classification', MultinomialNB())])

In [33]:
pipeline_clf.predict(documents)

array([3, 1], dtype=int64)

### Évaluation des Performances du Modèle

> Question 9 : Chargez le sous-ensemble de test et évaluez les performances du modèle en termes de précision moyenne.

Affichez également le rapport de classification et la matrice de confusion pour une évaluation plus détaillée.


In [34]:
dataset_test = fetch_20newsgroups(subset="test", categories=categories, shuffle=True, random_state=0)

y_pred = pipeline_clf.predict(dataset_test.data)

In [40]:
from sklearn import metrics

In [42]:
(y_pred == dataset_test.target).mean()

print(
    metrics.classification_report(dataset_test.target, y_pred, target_names=dataset_test.target_names)
)

                        precision    recall  f1-score   support

           alt.atheism       0.97      0.60      0.74       319
         comp.graphics       0.96      0.89      0.92       389
               sci.med       0.97      0.81      0.88       396
soc.religion.christian       0.65      0.99      0.78       398

              accuracy                           0.83      1502
             macro avg       0.89      0.82      0.83      1502
          weighted avg       0.88      0.83      0.84      1502



In [43]:
metrics.confusion_matrix(dataset_test.target, y_pred)

array([[192,   2,   6, 119],
       [  2, 347,   4,  36],
       [  2,  11, 322,  61],
       [  2,   2,   1, 393]], dtype=int64)

### Optimisation des Paramètres avec Grid Search

Scikit-Learn permet d'optimiser les paramètres des composants du pipeline par une recherche en grille.

> Question 10 : Utilisez `GridSearchCV` pour trouver les meilleurs paramètres parmi différents modèles de machine learning.

Testez les modèles `MultinomialNB` et `SGD`, avec une recherche de paramètres pour le modèle `SGD`.


In [46]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier

parameters = {
    "classification": [MultinomialNB(), SGDClassifier(max_iter=500, tol=1e-3)],
    "classification__alpha": (1e-2, 1e-3),
}

gridsearch_clf = GridSearchCV(pipeline_clf, parameters, cv=5)
gridsearch_clf.fit(dataset.data, dataset.target)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vectorisation', TfidfVectorizer()),
                                       ('classification', MultinomialNB())]),
             param_grid={'classification': [MultinomialNB(alpha=0.001),
                                            SGDClassifier(max_iter=500)],
                         'classification__alpha': (0.01, 0.001)})

> Question 11 : Affichez les meilleurs paramètres trouvés et la précision obtenue pour ces paramètres optimaux.


In [47]:
print(gridsearch_clf.best_params_)
print(gridsearch_clf.best_score_)

{'classification': MultinomialNB(alpha=0.001), 'classification__alpha': 0.001}
0.9747434413201734


 🎉